'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
   
PYTHON VERSION: 3.6.3  
SQLALCHEMY VERSION: 1.1.13

'''

### Pollutant Constituents
Below are the pollutant constituents we attempt to address through this alternatives analysis

In [1]:
#############################################################################################################
#                   
#                                       DEFINE GLOBAL VARIABLE pollLS
#############################################################################################################     
pollLS = ['tss', 'turbidity', 'p', 'n', 'nn', 'an', 'og', 'cu', 'zn', 'fe', 'phmin', 'phmax'] 

# Program Setup
## (Importing libraries, defining database)

In [2]:
#import standard python libraries:
import winsound
import pandas as pd
import numpy as np
import math
import datetime
import calendar

In [3]:
#IMPORT AND DEFINE sqlalchemy libraries, tables, and session engine
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import and_ #used in query.filter() to joing multiple where clauses
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#Table definitions as SQLA classes:
from SQLA_DB_base_bmp_feasibility_test_results import Base_BMP_Feasibility_Test_Results as BBFTR
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions as BBFTD
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_combo_bmps import Combo_BMPs
from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_type_has_nel import Facility_Type_Has_NEL
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions as FTQ
from SQLA_DB_nel_sample_classes import NEL_Sample_Classes
from SQLA_DB_existing_pollutant_concentrations import Existing_Pollutant_Concentrations as ExPollConcs
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates as PRR
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create SQLA classes

'''
Dictionary of "SQLAlchemy where clause lambda functions" that importCSV uses to test record uniqueness.
used as the where clause in sqlalchemy queries, updates and deletes 
Form:
    {TableName:Lambda Function, TableName:Lambda Function, ...}
    
    TableName is the table name we want to define uniqueness test for
    Lambda Function is a SQLAlchemy query used to test record uniqueness. The function can take on any form 
        but must be made to evaluate the CSV row passed as a dictionary (CSVRowDict in this explanation):
        CSVRowDict: {FieldName:CSVColValue, DBTableFieldName:CSVColValue...} 
            Where: DBTableFieldName is the name of the field associated with the value at CSVColValue on the current row
                   CSVColValue: a value in the CSV's current row+column corresponding to the DBTableFieldName 
        *this assumes that field names are unique across table. if not, then method fails (maybe need to extend method?)
    FALSE: indicates that db table doesn't impose uniqueness on its records (other than its record id being unique)
        
e.g.: lambda myRowVal: Base.metadata.tables['people'].c['name'] == CSVRowDict['name']
        using lambda function in query will search for CSVRowDict's value for 'name' in the table people, field name 
if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'facility_chars': lambda CSVRowDict: Base.metadata.tables['facility_chars'].c['Fac_Name'] == CSVRowDict['Fac_Name'],
    'facility_monthly_rain': False, #DB schema does not impose uniqueness on records in this table
    'facility_type_has_nel': False,
    'facility_risks': False,
    'facility_types': lambda CSVRowDict: Base.metadata.tables['facility_types'].c['Fac_Type'] == CSVRowDict['Fac_Type'],
    'nel_sample_classes': lambda CSVRowDict: Base.metadata.tables['nel_sample_classes'].c['nel_column']==CSVRowDict['nel_column'],
    'existing_pollutant_concentrations': False, #uniqueness not imposed for records in this table.
    'wrs_pollutant_risks': False #DB schema does not impose uniqueness on records in this table
}

import SQLA_main as SQLA_main #import main SQLAlchemy functions


Clearing old DB


In [4]:
'''
Define other custom modules
'''
import mod_Base_BMP_Eval as BBMP_Eval
import mod_Combo_BMP_Eval as CBMP_Eval
import mod_EffluentLimit as EffLim
import mod_expression as Expr
import mod_importSpecial as importSpecial #special import functions are defined here
import mod_importCSV as importCSV #generic CSV importer ****IMPORTANT NOTE: function assumes csv in the utf-8-sig file format. weird things happen if its not in this format!!!


#  Import Data

In [5]:
#import feasibillity questions, build feasibility expressions
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') 

#import base bmp information including:
  #1. imports definitions for cip costs, o&m costs, and BMP sizing to the expressions table
  #2. imports pollutant removal rates into pollutant_removal_rates table
  #3. creates a record in the base_bmps table using (1) and (2)
  #4. feasibility tests
importSpecial.importBaseBMPsCSV('Input_Files\\bmp_lego_piece.csv') 

#IMPORT BASIC FACILITY CHARS:
    #!!!!IMPORTANT!!!! This import must occur before other facility specific data is imported!
print ('\nImporting facility characteristics:')
importCSV.importCSV('Input_Files\\facility_chars.csv', unqTests)

#IMPORT PBP Appendix A1 data
print ('\nImporting PBP Appendix A1 data:')
importCSV.importCSV('Input_Files\\pbp_appxa1.csv', unqTests)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importCSV.importCSV('Input_Files\\FacilityRainfallData.csv', unqTests)

#IMPORT EFFLUENT LIMITS EXISTANCE FOR FACILITY TYPES: (either by Priority Based Plan, Table 3 or as City operational assignment)
#IF CSV HEADRS SETUP CORRECTLY, THEN THIS INSERTS NEL EXISTANCE DATA (0 OR 1) TO WRS_POLLUTANT TABLE 
#AND USES THE FACILITY_TYPE_HAS_NEL TO ASSOCIATE RECORD WITH FACILITY TYPE
print ('\nImporting Facility Type Has Effluent Limits:') #import into wrs_pollutant_risks table
importCSV.importCSV('Input_Files\\nel_exists_facility_types.csv', unqTests)

#IMPORT NEL CLASSIFICATION DATA (from PBP Appendix L)
print ('\nImporting NEL Classes')
importCSV.importCSV('Input_Files\\nel_pbp_appxl.csv', unqTests)

#IMPORT FACILITY RISKS:
print ('\nImporting Facility Risks')
#for future implementation:
    #The current process inserts fac risk and update existing_fac_char_id in Facility_chars table. this process thus creates
#dead records. a more sophisticated approach using sophisticated lambda function in unqTests would fix this
importCSV.importCSV('Input_Files\\facility_risks.csv', unqTests)

# #IMPORT FACILITY SAMPLING DATA
 #!!!IMPORTANT!!!! For now, we make none detects = 0 BUT this must be changed to detection limit, per DOH guidance.
print ('\nImporting Facilty Sampling data:')
importCSV.importCSV('Input_Files\\sample_data.csv', unqTests)


# for now, since we're developing, delete out all except 1st 2 facilities.
# n = 5
# session.query(ExPollConcs).filter(ExPollConcs.facility_id >n).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
# session.query(Facility_Chars).filter(Facility_Chars.id >n).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
# session.commit #we chose not to sync session so need to commit before proceeding to requery or else you may get unpredictable resutls

session.commit()
winsound.Beep(250,1000)

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1

Reading csv record: Feas-2

Reading csv record: Feas-3

Reading csv record: Feas-4

Reading csv record: Feas-5

Reading csv record: Feas-6

Reading csv record: Feas-7

Reading csv record: Feas-8

Reading csv record: Feas-9

Reading csv record: Feas-10

Reading csv record: Feas-11

Reading csv record: Feas-12

Reading csv record: Feas-13

Reading csv record: Feas-14

Reading csv record: Feas-15

Reading csv record: Feas-16

Reading csv record: Feas-17

Reading csv record: Feas-18

Reading csv record: Feas-19

Reading csv record: Feas-20

Reading csv record: Feas-21
Reading csv for import to base bmp tables

Reading csv record: Hydrodynamic Separation
Reading pollutant removal rate info...
Linking feasibility tests w/ base bmp: 1
Removed:  0  old feasibility test defs for the base bmp
Added feasibility test def as record:  1
Added feasibility test def as record:  2

Reading csv record: Enhanced Media Filtration 

# Existing Sampling Data
Talk about it...

Global variables related to existing sampling data include:  
 - 
 - 

Defined several functions that will be used by BMP Option Evaluation. These include:  
 - 
 - 

In [6]:
'''
#############################################################################################################
#              ASSIGN CONCENTRATION DATA FOR FACILITIES WITHOUT SAMPLING RESULTS:
#                      assignment made into database table: ExPollConcs 
#############################################################################################################
Enter estimated pollutant concentrations into database's existing pollutant concentration table for facilities without 
actual sampling data. Use 1 of 2 methods:

Method 1 (sim_MaxType): Use maximum concentration value sampled for period 2013-2017
          This method is for Permit Table 1 facilities only
          Method assumes we have already entered sampling data for into the database's existing pollutant concentration table

Method 2 (sim_EMC): Use data from an EMC study.
          This method is for facilities that are not on Permit Table 1
'''

def WriteSampleDat_simMaxType(pollLS):
    #assign maximum sampled values to Table 1 facilities that have not yet been sampled
    #delete all pollutant concentration table records that are not from infield sampling.
    #     To be sure we're starting fresh, let's remove any records in ExPollConcs that:
    #     1. Were not obtained directly from field samples (i.e. sample_method != 'infield)
    #     2. Were obtained from field samples, but are not Table 1 facilities (i.e. we shouldn't be looking at their  sample results)
    session.query(ExPollConcs).filter(ExPollConcs.sample_method != 'infield').delete(synchronize_session = False)
    #delete all pollutant concentration table records that are not for Table 1 facilities
    #for some reason bulk delete's not working. so let's use a loop to work around it.
    for rec in session.query(ExPollConcs.id).filter(ExPollConcs.facility_id == Facility_Chars.id).filter(Facility_Chars.Permit_Table != 'Table 1'):
        session.query(ExPollConcs).filter(ExPollConcs.id == rec[0]).delete(synchronize_session = False)

    #make a dataframe called pd_Concs to hold existing pollutant concentrations that were sampled in the field (the 'infield' sampling method)'''
    q = session.query(ExPollConcs).filter(ExPollConcs.sample_method == 'infield')
    pd_Concs = pd.read_sql(q.statement,session.bind)         

    #build pd_infieldExtreama by making a dictionary of maximum sample results for each constiuent
    dict_extrema = {'c_' + Constituent: pd_Concs.loc[:,'c_' + Constituent].max() for Constituent in pollLS}
    dict_extrema['c_phmin'] = pd_Concs.loc[:,'c_phmin'].min() #phMin is exception to above. we want min. phMin value
    #use dictionary to build pd_infieldExtrema dataframe
    pd_infieldExtrema = pd.DataFrame([dict_extrema])
    #     display(pd_infieldExtrema)

    #now build query that identifies all Table 1 facilities that are not in ExPollConcs
    subq = session.query(ExPollConcs.facility_id.distinct()).order_by(ExPollConcs.facility_id).all()
    ls_sq = [i[0] for i in subq if i[0] is not None] #list comprehension to produce list of all facility_id in ExPollConcs table
    #get list of Table 1 facilities not in ExPollConcs:
    tpl_q = session.query(Facility_Chars.id).filter(Facility_Chars.Permit_Table == 'Table 1').filter(Facility_Chars.id.notin_(ls_sq)).all()
    ls_FacIDs = [i[0] for i in tpl_q] #write query tuple to list    
    #make a list of Table 1 facs not in ExPollConcs (a list of dicts). also include extrema conc. values.  
    ls_dict_pd = [{**{'facility_id': FacID, 'sample_method': 'sim_MaxType', 'sample_date':'12/31/2016'}, **dict_extrema} for FacID in ls_FacIDs]
    #write list to database:
    ExPollConcs_meta = Base.metadata.tables['existing_pollutant_concentrations']
    ExPollConcs_id_meta = ExPollConcs_meta.c['id']
    for dict_temp in ls_dict_pd:
        SQLA_main.insertRec(ExPollConcs_meta,dict_temp)
    session.commit()
    #for future implementation: write dict -> dataframe -> db(using sqla):
        # pd_temp.to_sql('existing_pollutant_concentrations', engine, if_exists='append', index = False)
        #http://docs.sqlalchemy.org/en/latest/faq/performance.html#i-m-inserting-400-000-rows-with-the-orm-and-it-s-really-slow
        #https://stackoverflow.com/questions/31997859/bulk-insert-a-pandas-dataframe-using-sqlalchemy

WriteSampleDat_simMaxType(pollLS) #call function defined above   

In [7]:
#############################################################################################################
#                                 Write all sampling data from database to pd_ExConcs
#                                       (DEFINE GLOBAL VARIABLE: pd_ExConcs)
#############################################################################################################    
#get all existing sampling data.
q = session.query(ExPollConcs.facility_id.label('Facility_ID'), ExPollConcs.sample_date, 
        ExPollConcs.c_tss,
        ExPollConcs.c_turbidity,
        ExPollConcs.c_p,
        ExPollConcs.c_n,
        ExPollConcs.c_nn,
        ExPollConcs.c_an,
        ExPollConcs.c_og,
        ExPollConcs.c_cu,
        ExPollConcs.c_zn,
        ExPollConcs.c_fe,
        ExPollConcs.c_phmin,
        ExPollConcs.c_phmax  
         ).order_by(ExPollConcs.facility_id) #.filter(ExPollConcs.facility_id == FacID)
pd_ExConcs = pd.read_sql(q.statement,session.bind) 
#tidy up the sampling data
from datetime import datetime
pd_ExConcs['sample_date'] = pd.to_datetime(pd_ExConcs['sample_date'], format="%m/%d/%Y")
#Write to a dataframe called pd_ExConcs
pd_ExConcs = pd_ExConcs.applymap(lambda x: float('nan') if x is None else x) #assign NaN values to any None element 
print ('a few pieces of data:')
pd_ExConcs.head(10)

a few pieces of data:


,Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax
0,1,2017-04-19,122.0,NaN,NaN,2.430,NaN,NaN,NaN,NaN,NaN,NaN,7.00,7.00
1,1,2017-02-11,59.0,13.0,0.097,0.580,NaN,NaN,NaN,NaN,NaN,NaN,8.30,8.30
2,1,2016-12-04,80.0,71.2,0.300,0.910,0.120,0.141,0.0,NaN,NaN,NaN,8.20,8.20
3,1,2016-06-17,83.0,81.4,0.250,0.940,0.200,0.060,0.0,NaN,NaN,NaN,6.92,6.92
4,1,2015-02-20,33.5,17.5,0.176,1.830,0.190,1.090,5.7,NaN,NaN,NaN,8.54,8.54
5,1,2014-04-13,14.0,8.5,0.244,2.037,0.247,0.556,4.7,NaN,NaN,NaN,6.64,6.64
6,1,2013-03-09,163.0,24.4,0.155,1.199,0.239,0.073,5.0,NaN,NaN,NaN,8.09,8.09
7,2,2017-04-20,NaN,13.0,NaN,0.780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,2017-01-21,0.0,31.0,0.100,1.580,0.000,0.418,0.0,NaN,NaN,NaN,7.20,7.20
9,2,2016-05-05,7.0,4.9,0.066,2.672,0.212,0.416,0.0,NaN,NaN,NaN,6.83,6.83


In [8]:
#############################################################################################################
#                                 ESTIMATE Numeric Effluent Limits
#                          (DEFINE GLOBAL VARIABLES: pd_FacsNELs_Wet & pd_FacsNELs_Dry)
#############################################################################################################    
'''
Estimate the Numeric Effluent Limits (NELs) for each facility.
Return wet and dry season NELs in 2 separate dataframes:
    pd_FacsNELs_Wet & pd_FacsNELs_Dry
Estimate NELs using the EffLim module's GetNELs function call.
 The GetNELs function call will differentiate between wet and dry season limits
 (if limits are the same between wet & dry season, then the same limit will be placed into the wet and dry
  dataframes.)
 The GetNEls function calculates a pollutant constituent NEL using this formula:
    NEL = fTypeHas_NEL * SampleClass_NEL
    Where:
      fTypeHas_NEL is a [0,1] value from PBP Table 3, based on facility type (stored in SQLA_DB_facility_type_has_nel)
      SampleClass_NEL is pollutant concentration based on facility's sample class, based on PBP Appendix L
'''
pd_FacsNELs_Wet, pd_FacsNELs_Dry = pd.DataFrame(),  pd.DataFrame() #initialize wet and dry season nel dataframes 
for recFac in session.query(Facility_Chars): #do the following for each facility:
    
    wet,dry = EffLim.GetNELs(recFac,False) #Get Wed & Dry NELs by calculating: NEL = fTypeHas_NEL * SampleClass_NEL
#     if wet is not None:
    pd_FacsNELs_Wet = pd.concat([pd_FacsNELs_Wet, wet]) #write wet NELs to pd_FacsNELs_Wet
#     if dry is not None:
    pd_FacsNELs_Dry = pd.concat([pd_FacsNELs_Dry, dry]) #write dry NELs to pd_FacsNELs_Dry

print('Wet NELs:')
display(pd_FacsNELs_Wet)
print('Dry NELs:')
display(pd_FacsNELs_Dry)

Wet NELs:


,wrs_tss,wrs_turbidity,wrs_p,wrs_n,wrs_nn,wrs_an,wrs_og,wrs_cu,wrs_zn,wrs_fe,wrs_phmin,wrs_phmax
Facility_ID,,,,,,,,,,,,
1,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
2,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
3,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,5.5,8.0
4,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
5,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
6,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
7,50.0,15.00,0.10,0.52,0.180,NaN,15.0,NaN,NaN,NaN,5.5,8.0
8,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,5.5,8.0
9,NaN,0.50,0.03,0.18,0.010,0.0050,15.0,NaN,NaN,NaN,7.6,8.6


Dry NELs:


,wrs_tss,wrs_turbidity,wrs_p,wrs_n,wrs_nn,wrs_an,wrs_og,wrs_cu,wrs_zn,wrs_fe,wrs_phmin,wrs_phmax
Facility_ID,,,,,,,,,,,,
1,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
2,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
3,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,5.5,8.0
4,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
5,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
6,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
7,30.0,5.50,0.06,0.38,0.090,NaN,15.0,NaN,NaN,NaN,5.5,8.0
8,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,5.5,8.0
9,NaN,0.50,0.03,0.18,0.010,0.0050,15.0,NaN,NaN,NaN,7.6,8.6


In [9]:
#############################################################################################################
#                               Estimate Exceedances of Faclility Effluent Limits
#                      
#############################################################################################################     
def CalcExceedances(pd_Concs, pollLS, pd_FacsNELs_Wet, pd_FacsNELs_Dry):
    '''    
    #for each facility in database, calculate exceedance for each pollutant constituent in pollLS list
    #do the Exceedance Calculation = max(0,(Constituent Concentration - NEL))
    # if no exceedance, then report 0. report NaN sample result is NaN
    #INPUT:
        pd_Concs: dataframe of concentrations [Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax]
        pollLS: list of pollutant constituents we want to analyze (constituent list needs to match those in pd_Concs and FacsNELs dataframes)
        pd_FacsNELs_Wet & pdFacsNELs_Dry: dataframes holding numerical effluent limits for each pollutant
    #Return dataframe [Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax]
    '''
    pd_FacExceedances = pd.DataFrame() #make an empty dataframe.  we will append to it.
    for Facs in session.query(Facility_Chars.id).order_by(Facility_Chars.id):
        FacID = Facs[0]
        if (FacID in pd_Concs.Facility_ID.values) and (FacID in pd_FacsNELs_Dry.index) and (FacID in pd_FacsNELs_Wet.index):
            pd_temp = pd_Concs.loc[pd_Concs['Facility_ID'] == FacID] #slice facility id rows into a temp dataframe
            pd_temp.is_copy = False #acknowledge that pd_temp is NOT a copy of pd_ExConcs. but intended to be treated as new dataframe
            for Constituent in pollLS:
                pd_temp['c_' + Constituent]  = pd_Concs.apply(lambda row: #for each row in pd_ExConcs:
                         EffLim.ExceedanceCalc(row, Constituent, FacID, pd_FacsNELs_Wet, pd_FacsNELs_Dry, True), axis = 1)
            pd_FacExceedances = pd.concat([pd_FacExceedances,pd_temp])
    return (pd_FacExceedances)
#############################################################################################################
#                          
#                                (DEFINE GLOBAL VARIABLE: pd_exFacExceedances)
############################################################################################################# 
pd_exFacExceedances = CalcExceedances (pd_ExConcs, pollLS, pd_FacsNELs_Wet, pd_FacsNELs_Dry)
print('Concentrations in excess of wet/dry season NELs')
display(pd_exFacExceedances)

Concentrations in excess of wet/dry season NELs


,Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax
0,1,2017-04-19,72.0,NaN,NaN,1.910,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
1,1,2017-02-11,9.0,0.0,0.000,0.060,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.30
2,1,2016-12-04,30.0,56.2,0.200,0.390,0.000,NaN,0.0,NaN,NaN,NaN,0.00,0.20
3,1,2016-06-17,53.0,75.9,0.190,0.560,0.110,NaN,0.0,NaN,NaN,NaN,0.00,0.00
4,1,2015-02-20,0.0,2.5,0.076,1.310,0.010,NaN,0.0,NaN,NaN,NaN,0.00,0.54
5,1,2014-04-13,0.0,0.0,0.144,1.517,0.067,NaN,0.0,NaN,NaN,NaN,0.00,0.00
6,1,2013-03-09,113.0,9.4,0.055,0.679,0.059,NaN,0.0,NaN,NaN,NaN,0.00,0.09
7,2,2017-04-20,NaN,0.0,NaN,0.260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,2017-01-21,0.0,16.0,0.000,1.060,0.000,NaN,0.0,NaN,NaN,NaN,0.00,0.00
9,2,2016-05-05,0.0,0.0,0.006,2.292,0.122,NaN,0.0,NaN,NaN,NaN,0.00,0.00


In [10]:
#############################################################################################################
#       CALCULATE EXISTING AGE FACTOR WEIGHTED AVERAGE FACILITY EXCEEDANCE VALUES FOR EACH CONSTITUENT:
#       
#############################################################################################################   
'''
Age factor acknowledges fact that more recent samples are a better representation of facility pollutant discharge 
(i.e. sampling data) and housekeeping-operations (i.e. inspections) realities. But, historic data as a whole also tells part 
of story (i.e. we want to dampen whipsaw effects that may occur if we only considered most recent data).

AF = exp(-SampleRank)
SampleRank = Newest sample = 1
              Second Newest sample = 2
              ...
              nth Newest Sample = n (out of n samples)
'''
def AFWFacExceedances(pd_FacExceedances, pollLS):
    #calculate age factor weighted averages for each constituent in pollLS FOR each facility IN DATABASE.
    #write these averages into a dataframe called pd_AFWFacExceedances [Facility_ID,sample_date,c_tss,c_turbidity,c_p,c_n,c_nn,c_an,c_og,c_cu,c_zn,c_fe,c_phmin,c_phmax]
    pd_AFWFacExceedances = pd.DataFrame() #make an empty dataframe.  we will append to it.
    ShowCalculations = False
    UnqFacIDs = pd_FacExceedances.Facility_ID.unique() #get unique facility IDs
    for FacID in UnqFacIDs:
        pd_temp = pd_FacExceedances.loc[pd_FacExceedances['Facility_ID'] == FacID].set_index('sample_date') #write current fac's data to temp df
        pd_AFWFacExceedances = pd.concat([pd_AFWFacExceedances,EffLim.Calc_AFWtdExceedances(pd_temp,pollLS,ShowCalculations)]) #calculate age factor weighted averages
    return pd_AFWFacExceedances

#############################################################################################################
#                        calculate age factor exceedances of existing samples in pd_exFacExceedances
#                                    DEFINE GLOBAL VARIABLE: pd_exAFWFacExceedances
#############################################################################################################    
pd_exAFWFacExceedances = AFWFacExceedances(pd_exFacExceedances, pollLS)
print ('Age Factor Weighted Averages:')
display(pd_exAFWFacExceedances)

Age Factor Weighted Averages:


,Facility_ID,AFWtd_c_tss,AFWtd_c_turbidity,AFWtd_c_p,AFWtd_c_n,AFWtd_c_nn,AFWtd_c_an,AFWtd_c_og,AFWtd_c_cu,AFWtd_c_zn,AFWtd_c_fe,AFWtd_c_phmin,AFWtd_c_phmax
0,1,52.064543,19.729711,0.067223,1.296165,0.029425,NaN,0.000000,NaN,NaN,NaN,0.000000,0.093351
0,2,0.961758,3.745947,0.004876,0.637805,0.031165,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000
0,3,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.118441
0,4,197.000000,0.000000,0.655000,0.829000,0.000000,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000
0,5,180.960566,199.986459,1.752511,17.437221,0.004532,NaN,1.421297,NaN,NaN,NaN,0.000000,0.000000
0,6,1960.000000,4166.000000,10.200000,26.730000,3.500000,NaN,44.000000,NaN,NaN,NaN,0.000000,0.700000
0,7,0.000000,12.800000,0.000000,0.323000,0.043000,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000
0,8,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.070000
0,9,NaN,4180.500000,10.270000,27.070000,3.670000,10.495000,44.000000,NaN,NaN,NaN,1.590000,0.100000
0,10,1960.000000,4166.000000,10.200000,26.730000,3.500000,NaN,44.000000,NaN,NaN,NaN,0.000000,0.700000


In [11]:
#############################################################################################################
#                       Estimate Exceedances of Faclility Effluent Limits
#                       DEFINE GLOBAL VARIABLE: pd_RunoffVols
#############################################################################################################   
#get facility imperviousness and area. order by Facility_ID so it's given in same order as monthly rain data dataframe
q_facDat = session.query(Facility_Chars.id.label('Facility_ID'), 
                         Facility_Chars.Indus_Area, 
                         Facility_Chars.Imperv.label('Imperv')).order_by('Facility_ID')
pd_facDat = pd.read_sql(q_facDat.statement,session.bind)

#get monthly rain data for each facility. order by facility_id so order matches facility data dataframe
q_rain = session.query(Facility_Chars.id.label('Facility_ID'), Facility_Monthly_Rain).filter(
    Facility_Chars.facility_monthly_rain_id == Facility_Monthly_Rain.id).order_by('Facility_ID')
pd_rainDat = pd.read_sql(q_rain.statement,session.bind)

#create a new dataframe to hold rain volumes
pd_RunoffVols = pd_facDat.loc[:,['Facility_ID']] #put facilities into the new dataframe
#now calculate volumes for each month:
for mo in range(1,13):
    pd_RunoffVols[calendar.month_name[mo]] = pd.DataFrame(pd_facDat['Indus_Area'] * pd_facDat['Imperv'] * pd_rainDat[calendar.month_name[mo]]/12)
#add monthlys together to get annual volume
pd_RunoffVols['Annual_Volume'] = pd_RunoffVols[[calendar.month_name[mo] for mo in range (1,13)]].sum(axis = 1)
display(pd_RunoffVols)

,Facility_ID,January,February,March,April,May,June,July,August,September,October,November,December,Annual_Volume
0,1,295325.500000,335399.166667,352822.500000,250024.833333,157681.166667,140257.833333,143742.500000,138515.500000,170748.666667,259607.666667,334528.000000,392025.000000,2.970678e+06
1,2,338916.586546,246040.406579,241152.186581,101837.916631,131167.236620,96134.993299,96949.696632,78211.519972,105096.729963,263149.176573,351951.839875,338101.883213,2.388710e+06
2,3,77993.831554,75165.810944,79184.577074,53434.705206,42122.622767,29322.108428,40038.818107,30810.540327,33638.560937,61918.767035,86924.422953,84691.775103,6.952465e+05
3,4,21601.669993,18133.109161,16552.499161,9439.754164,7771.332498,6454.157498,7376.179998,7595.709164,9878.812497,13698.619996,17298.898328,18177.014994,1.539778e+05
4,5,50545.681653,57766.493317,56824.648318,36261.032490,25272.840826,21191.512494,21348.486661,23232.176660,26528.634159,43795.792488,56039.777485,65615.201649,4.844223e+05
5,6,15974.190828,18427.906661,18127.451661,11667.669163,7911.981664,6810.313331,6910.464998,7311.071664,8462.815831,13871.005829,18027.299994,20831.546660,1.543337e+05
6,7,10058.130000,7407.150000,7069.280000,2858.900000,3742.560000,2521.030000,2754.940000,2365.090000,3066.820000,7407.150000,10447.980000,9928.180000,6.962721e+04
7,8,31502.679978,29712.754979,32397.642477,21658.092485,17183.279988,11873.169158,16168.989155,12529.474991,13603.429990,24999.285815,35142.194142,34426.224142,2.811972e+05
8,9,9823.037518,8034.365015,6568.240012,3782.602507,2140.542504,967.642502,1554.092503,3548.022506,3108.185006,6978.755013,7565.205014,9529.812517,6.360050e+04
9,10,7359.825004,4947.100003,3973.900002,2108.600001,1581.450001,648.800000,811.000000,3000.700002,3467.025002,4460.500002,5170.125003,5920.300003,4.344933e+04


In [20]:
#############################################################################################################
#                       Calculate raw pollutant exceedance potential scores (PEP_raw)
#                         PEP_raw = AFWtd Exceedance * Annual Runoff Volume (cu. ft)
############################################################################################################   
def _HELPER_calc_PEP_raw(row, Constituent, pd_RunoffVols):
    #HELPER function to calculate PEP_raw
    AnnRunoffVol = pd_RunoffVols.loc[pd_RunoffVols['Facility_ID']==row.loc['Facility_ID'],'Annual_Volume'].values[0]
    AFWFacExceedVal = row.loc['AFWtd_c_'+Constituent]
    return  AFWFacExceedVal * AnnRunoffVol
def CalcPEP_Raw(pd_AFWFacExceedances,pollLS, pd_RunoffVols):
    #use age factor weighted scores to calculate raw PEP scores for each constituent pollutant
    #input: 
        #pd_AFWFacExceedances: [Facility_ID	AFWtd_c_tss	AFWtd_c_turbidity	AFWtd_c_p	AFWtd_c_n	AFWtd_c_nn	AFWtd_c_an	AFWtd_c_og	AFWtd_c_cu	AFWtd_c_zn	AFWtd_c_fe	AFWtd_c_phmin	AFWtd_c_phmax]
        #pollLS: pollutant constituent list
        #pd_RunoffVols: RUNOFF VOLUMES [Facility_ID	January	February	March	April	May	June	July	August	September	October	November	December	Annual_Volume] 
    #output: pd_PEP_raw[	Facility_ID	PEP_raw_tss	PEP_raw_turbidity	PEP_raw_p	PEP_raw_n	PEP_raw_nn	PEP_raw_an	PEP_raw_og	PEP_raw_cu	PEP_raw_zn	PEP_raw_fe	PEP_raw_phmin	PEP_raw_phmax]

    #initialize pd_PEP_raw dataframe w/ Facility_IDs from pd_AFWFacExceedances
    pd_PEP_raw = pd_AFWFacExceedances.loc[:,['Facility_ID']]
#     display(pd_PEP_raw)
#     pd_PEP_raw.reset_index(drop=True)
    #for each facility in pd_exPEP_raw, calculate PEP_Raw SCORE for each pollutant constituent in the pollLS LIST:
    for Constituent in pollLS:
        pd_PEP_raw['PEP_raw_' + Constituent] = pd_AFWFacExceedances.apply(lambda row: 
                                               _HELPER_calc_PEP_raw(row,Constituent, pd_RunoffVols), axis = 1)    
    return pd_PEP_raw

#############################################################################################################
#                              calculate existing PEP_raw scores
#                              DEFINE GLOBAL VARIABLE: pd_exPEP_raw
#############################################################################################################  

pd_exPEP_raw = CalcPEP_Raw(pd_exAFWFacExceedances,pollLS,pd_RunoffVols)
display(pd_exPEP_raw)

,Facility_ID,PEP_raw_tss,PEP_raw_turbidity,PEP_raw_p,PEP_raw_n,PEP_raw_nn,PEP_raw_an,PEP_raw_og,PEP_raw_cu,PEP_raw_zn,PEP_raw_fe,PEP_raw_phmin,PEP_raw_phmax
0,1,1.546670e+08,5.861063e+07,1.996976e+05,3.850488e+06,8.741303e+04,NaN,0.000000e+00,NaN,NaN,NaN,0.000000,277315.755592
0,2,2.297361e+06,8.947981e+06,1.164828e+04,1.523531e+06,7.444524e+04,NaN,0.000000e+00,NaN,NaN,NaN,0.000000,0.000000
0,3,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.000000,82345.979883
0,4,3.033362e+07,0.000000e+00,1.008554e+05,1.276476e+05,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,0.000000,0.000000
0,5,8.766133e+07,9.687790e+07,8.489555e+05,8.446978e+06,2.195162e+03,NaN,6.885079e+05,NaN,NaN,NaN,0.000000,0.000000
0,6,3.024941e+08,6.429543e+08,1.574204e+06,4.125340e+06,5.401680e+05,NaN,6.790684e+06,NaN,NaN,NaN,0.000000,108033.602799
0,7,0.000000e+00,8.912283e+05,0.000000e+00,2.248959e+04,2.993970e+03,NaN,0.000000e+00,NaN,NaN,NaN,0.000000,0.000000
0,8,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,0.000000,19683.805211
0,9,NaN,2.658819e+08,6.531772e+05,1.721666e+06,2.334138e+05,6.674873e+05,2.798422e+06,NaN,NaN,NaN,101124.799158,6360.050261
0,10,8.516068e+07,1.810099e+08,4.431831e+05,1.161400e+06,1.520726e+05,NaN,1.911770e+06,NaN,NaN,NaN,0.000000,30414.527517


In [22]:
#############################################################################################################
#                       Calculate normalized pollutant exceedance potential scores (PEP_norm)
#                         PEP_Norm = (PEP_raw - PEPmin) / (PEPMax - PEPmin)
############################################################################################################   
'''
NORMALIZE the raw Pollutant Exceedance Potential scores held in a pd_PEP_raw dataframe to a new dataframe called pd_PEP_norm.
Use calculation:
PEP_Norm = (PEP_raw - PEPmin) / (PEPMax - PEPmin)

Hold the PEPmax and PEPmin baseline scores used for the normalization in a dataframe called pd_NormBaselinePEP
****NOTE: LATER, we'll need to write the norm. basis to file
          This will allow us to use a common baseline in future (when we get more data, we'll want to have same baseline)         
'''
#############################################################################################################
#                           BUILD BASELINE dataframe pd_NormBaselinePEP
#                              DEFINE GLOBAL VARIABLE: pd_NormBaselinePEP
############################################################################################################  
##Use pd_exPEP_Ras data as our baseline max. Use 0 as min for all:
dict_NormBaselinePEP = {'PEP_Baseline_' + Constituent: [pd_exPEP_raw.loc[:,'PEP_raw_' + Constituent].max(),
                                             0]
                                             for Constituent in pollLS}
dict_NormBaselinePEP['MaxMin'] = ['Max','Min'] #add column identifying if row is max or min
pd_NormBaselinePEP = pd.DataFrame(dict_NormBaselinePEP) #write dict to new dataframe 
print ('This is the pd_NormBaselinePEP dataframe:')
display(pd_NormBaselinePEP)

#############################################################################################################
#                                        CALCULATE PEP_norm
#                         
############################################################################################################ 
def _HELPER_calc_PEP_norm(row, Constituent, pd_NormBaselinePEP):
    #HELPER function to calculate norm of PEP_raw for a given constituent
    #get max and min baseline values from pd_NormBAselinePEP
    BLmax= pd_NormBaselinePEP.loc[pd_NormBaselinePEP['MaxMin']=='Max', 'PEP_Baseline_' + Constituent].values[0] 
    BLmin= pd_NormBaselinePEP.loc[pd_NormBaselinePEP['MaxMin']=='Min', 'PEP_Baseline_' + Constituent].values[0]
    #i dont know how to catch div by zero errors using numpy :-/
    dv = row['PEP_raw_' + Constituent] - BLmin
    if dv != 0:
        v = (dv - BLmin)/(BLmax - BLmin)
    else:
        v = 0 #force to 0
    return v
def CalcPEP_norm(pd_PEP_raw,pollLS, pd_NormBaselinePEP):
    #calculate PEP_norm for each constituent pollutant of each facility in pd_PEP_raw
    #return pd_PEP_norm [	Facility_ID	PEP_norm_tss	PEP_norm_turbidity	PEP_norm_p	PEP_norm_n	PEP_norm_nn	PEP_norm_an	PEP_norm_og	PEP_norm_cu	PEP_norm_zn	PEP_norm_fe	PEP_norm_phmin	PEP_norm_phmax]
    
    #initialize pd_PEP_norm dataframe w/ Facility_IDs from pd_exPEP_raw
    pd_PEP_norm = pd_PEP_raw.loc[:,['Facility_ID']]
#     pd_PEP_norm.reset_index(drop=True,inplace=True)
    #loop through each constituent; calculating norms as we go:   
    for Constituent in pollLS:
            pd_PEP_norm['PEP_norm_' + Constituent] = pd_PEP_raw.apply(lambda row: 
                                               _HELPER_calc_PEP_norm(row,Constituent,pd_NormBaselinePEP), axis = 1)
    return pd_PEP_norm

#############################################################################################################
#                       Normalize existing raw pollutant exceedance potential scores 
#                         (DEFINE GLOBAL VARIABLE: pd_exPEP_norm)
############################################################################################################   
print('This is the pd_exPEP_norm dataframe:')
pd_exPEP_norm = CalcPEP_norm(pd_exPEP_raw,pollLS,pd_NormBaselinePEP)
display(pd_exPEP_norm)

#TO DO:  WRITE existing NORMALIZED PEP SCORES TO DB: 

This is the pd_NormBaselinePEP dataframe:


,MaxMin,PEP_Baseline_an,PEP_Baseline_cu,PEP_Baseline_fe,PEP_Baseline_n,PEP_Baseline_nn,PEP_Baseline_og,PEP_Baseline_p,PEP_Baseline_phmax,PEP_Baseline_phmin,PEP_Baseline_tss,PEP_Baseline_turbidity,PEP_Baseline_zn
0,Max,7.452262e+06,2.297250e+08,4.448060e+09,1.911565e+07,2.600866e+06,3.126724e+07,7.283846e+06,497346.455645,703512.890691,1.392570e+09,2.968967e+09,1.051438e+09
1,Min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00


This is the pd_exPEP_norm dataframe:


,Facility_ID,PEP_norm_tss,PEP_norm_turbidity,PEP_norm_p,PEP_norm_n,PEP_norm_nn,PEP_norm_an,PEP_norm_og,PEP_norm_cu,PEP_norm_zn,PEP_norm_fe,PEP_norm_phmin,PEP_norm_phmax
0,1,0.111066,0.019741,0.027417,0.201431,0.033609,NaN,0.000000,NaN,NaN,NaN,0.000000,0.557591
0,2,0.001650,0.003014,0.001599,0.079701,0.028623,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000
0,3,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.165571
0,4,0.021782,0.000000,0.013846,0.006678,0.000000,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000
0,5,0.062949,0.032630,0.116553,0.441888,0.000844,NaN,0.022020,NaN,NaN,NaN,0.000000,0.000000
0,6,0.217220,0.216558,0.216123,0.215810,0.207688,NaN,0.217182,NaN,NaN,NaN,0.000000,0.217220
0,7,0.000000,0.000300,0.000000,0.001177,0.001151,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000
0,8,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.039578
0,9,NaN,0.089554,0.089675,0.090066,0.089745,0.089568,0.089500,NaN,NaN,NaN,0.143743,0.012788
0,10,0.061154,0.060967,0.060845,0.060757,0.058470,NaN,0.061143,NaN,NaN,NaN,0.000000,0.061154


In [23]:
#############################################################################################################
#                                        Sum Normalized PEP Scores
#                                write scores to new dataframe called pd_PEP_sum
############################################################################################################ 
def SumNormPEPs (pd_PEP_norm):
    #general function to sum normalized PEPs
    pd_PEP_norm.set_index('Facility_ID', inplace=True) #move FAcility ID to index temporarily
    pd_PEP_norm['PEP_norm_sum'] = pd_PEP_norm.sum(axis = 1) #sum norm scores for each facility
    pd_PEP_norm.reset_index(inplace=True) #move facility ID from index
    return(pd_PEP_norm.loc[:,['Facility_ID', 'PEP_norm_sum']]) #copy summed norm scores and return dataframe

#############################################################################################################
#                                  Sum existing Normalized PEP Scores
#                              (DEFINE GLOBAL VARIABLE: pd_exPEP_sum)
############################################################################################################ 
pd_exPEP_sum = SumNormPEPs(pd_exPEP_norm)
display(pd_exPEP_sum)

,Facility_ID,PEP_norm_sum
0,1,0.950855
1,2,0.114587
2,3,0.165571
3,4,0.042307
4,5,0.676885
5,6,1.507800
6,7,0.002628
7,8,0.039578
8,9,0.694638
9,10,0.424489


In [48]:
'''
#############################################################################################################
#                               CALCULATE WRS PEP BASE SCORES
#                      WRS PEP BASE SCORE = NORM_PEP_SCORE*(SampleUncertainty + 1) 
############################################################################################################ 
'''
def _HELPER_PEPUncertainty(ls_id, dict_unc):
    '''determine the uncertainty level based on sample method
        (retrieve list of sample methods from ExPollConcs table for facilities in ls_id; assign uncertainty level using dict_unc)
       input: 
            ls_id: list of facility ids
            dict_unc: dictionary of uncertainty values for each sample method
        return: 
            pd_unc: dataframe [Facility_ID, UncertaintyValue]
    '''
    #get sample method for each facility in ls_id list
    q = session.query(ExPollConcs.facility_id.label('Facility_ID'), ExPollConcs.sample_method.label('sample_method')).filter(
        ExPollConcs.facility_id.in_(ls_id)).distinct(ExPollConcs.facility_id).order_by(ExPollConcs.facility_id)
    pd_samplemethod = pd.read_sql(q.statement,session.bind)
    #use dict_unc to assign uncertainty value for each facility's sample method
    pd_samplemethod['Uncertainty_Value'] = pd_samplemethod['sample_method'].apply(lambda val: dict_unc[val])
    return pd_samplemethod
    
def CalcWRSPEPBaseScore(pd_PEP_sum):
    #calculate wrs pep base score = NORM_PEP_SCORE*(SampleUncertainty + 1) 
    #input: pd_PEP_sum dataframe containing COLUMNS [Facility_ID, PEP_norm_sum]
    #return: dataframe of WRS PEP Base Scores
    
    #make a list of Facility IDs in pd_PEP_sum
    ls_id = [np.asscalar(id) for id in pd_PEP_sum['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
    #make the pd_WRSPEPBaseScore dataframe:
    #write uncertainty information into pd_WRSPEPBaseScore
    pd_WRSPEPBaseScore = _HELPER_PEPUncertainty(ls_id, {'infield':0.25, 'sim_MaxType':1.0, 'sim_EMC':0.0})
    #copy in PEP_norm_sum values
    pd_WRSPEPBaseScore['PEP_norm_sum'] = pd_PEP_sum['PEP_norm_sum']
    #calculate PEP wrs and then write result into column
    pd_WRSPEPBaseScore['PEP_BaseRisk'] = pd_WRSPEPBaseScore['PEP_norm_sum'] * (pd_WRSPEPBaseScore['Uncertainty_Value'] + 1)
    return pd_WRSPEPBaseScore

#############################################################################################################
#                            CALCULATE existing WRS PEP BASE SCORES & (TO DO: WRITE SCORES TO database) 
#                              (DEFINE GLOBAL VARIABLE: pd_exWRSPEPBaseScore)
############################################################################################################ 
#calc WRS PEP Base Scores for existing normalized PEP sums (pd_exPEP_norm)
pd_exWRSPEPBaseScore = CalcWRSPEPBaseScore(pd_exPEP_sum)
display(pd_exWRSPEPBaseScore)

,Facility_ID,sample_method,Uncertainty_Value,PEP_norm_sum,PEP_BaseRisk
0,1,infield,0.25,0.950855,1.188568
1,2,infield,0.25,0.114587,0.143233
2,3,infield,0.25,0.165571,0.206963
3,4,infield,0.25,0.042307,0.052883
4,5,infield,0.25,0.676885,0.846106
5,6,sim_MaxType,1.00,1.507800,3.015601
6,7,infield,0.25,0.002628,0.003285
7,8,infield,0.25,0.039578,0.049472
8,9,sim_MaxType,1.00,0.694638,1.389276
9,10,sim_MaxType,1.00,0.424489,0.848977


In [25]:
'''
#############################################################################################################
#                                    CALCULATE WRS BASE SCORES  
#
############################################################################################################ 
CALCULATE WRS BASE SCORE:
    TABLE 1 Facilities: WRS_BASE = WRS_INHERENT + WRS_CONTROLLABLE
        WRS_CONTROLLABLE = WRS_BMP + WRS_PEP
        WRS_BMP = WRS_HOUSEKEEPING + WRS_PCBMP
    TABLE 1A Facilities: 
    TABLE 2 & non-permitted:

    INPUTS:
        pd_wrsNonPEPScores
        pd_wrsPEPScores
'''
def GET_pd_FacRisks(ls_id):
    '''helper function that takes in list of facility_char ids and returns dataframe of:
        Facility_ID
        Inherent base risk
        housekeeping bmp base risk
        sw plan base risk
        bmp inspection deficiency rate (pc base risk)
    
        these items are obtained by querying database table: Facility Risks
    '''    
    q_facriskIDs =  session.query(Facility_Chars.existing_facility_risk_id).filter(Facility_Chars.id.in_(ls_id)) #for facilities in pd_exPEP_sum, get existing_facility_risk_id records
    #use q_facriskIDs as filter on Facility_Risks table to get associated wrs pollutant base id
    q_facrisks = session.query(
        Facility_Chars.id.label('Facility_ID'),Facility_Risks.Category_RiskFactor, Facility_Risks.Inherent_BaseRisk, Facility_Risks.HousekeepingBMP_BaseRisk, Facility_Risks.SWPlan_BaseRisk, Facility_Risks.BMPInspectionDeficiency_Rate).filter(
            Facility_Risks.id.in_(q_facriskIDs)).filter(
                Facility_Risks.id == Facility_Chars.existing_facility_risk_id).order_by(Facility_Chars.id)
    pd_facrisks = pd.read_sql(q_facrisks.statement, session.bind)
    return pd_facrisks

def CalcWRSBaseScore(pd_wrsNonPEPScores, pd_wrsPEPScores):
    '''
    CALCULATE WRS BASE SCORE:
        TABLE 1 Facilities: WRS_BASE = WRS_INHERENT + WRS_CONTROLLABLE
            WRS_CONTROLLABLE = WRS_BMP + WRS_PEP
            WRS_BMP = WRS_HOUSEKEEPING + WRS_PCBMP
        TABLE 1A Facilities: 
        TABLE 2 & non-permitted:

        INPUTS:
            pd_wrsNonPEPScores [Facility_ID	Category_RiskFactor	Inherent_BaseRisk	HousekeepingBMP_BaseRisk	SWPlan_BaseRisk	BMPInspectionDeficiency_Rate]
            pd_wrsPEPScores [	Facility_ID	sample_method	Uncertainty_Value	PEP_norm_sum	PEP_BaseRisk ]

    '''
    #initialize pd_exPEP_norm dataframe w/ Facility_IDs from pd_exPEP_raw
    pd_wrsBaseScores = pd.merge(pd_wrsNonPEPScores, pd_wrsPEPScores, on='Facility_ID')
    
    #calculate Table 1 scores (no need to differentiate tables now. all facilities are table 1)
    pd_wrsBaseScores['BMP_BaseRisk'] = pd_wrsBaseScores['HousekeepingBMP_BaseRisk'] + pd_wrsBaseScores['BMPInspectionDeficiency_Rate']
    pd_wrsBaseScores['Controllable_BaseRisk'] = pd_wrsBaseScores['BMP_BaseRisk'] + pd_wrsBaseScores['PEP_BaseRisk']
    #calculate total score:
    pd_wrsBaseScores['BMP_Total'] = pd_wrsBaseScores['Inherent_BaseRisk'] + pd_wrsBaseScores['Controllable_BaseRisk']
    return pd_wrsBaseScores

#############################################################################################################
#                             CALCULATE existing WRS BASE SCORES 
#                       (DEFINE GLOBAL VARIABLE: pd_exwrsNonPEPScores & pd_exwrsBaseScores)
############################################################################################################ 
#make a list of Facility IDs in pd_exWRSPEPBaseScore
ls_id = [np.asscalar(id) for id in pd_exWRSPEPBaseScore['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
#get nonPEP WRS scores for each facility
pd_exwrsNonPEPScores = GET_pd_FacRisks(ls_id)
#make base scores using existing sub-scores.
pd_exwrsBaseScores =  CalcWRSBaseScore(pd_exwrsNonPEPScores,pd_exWRSPEPBaseScore.loc[:,['Facility_ID','PEP_BaseRisk']])
display(pd_exwrsBaseScores)

,Facility_ID,Category_RiskFactor,Inherent_BaseRisk,HousekeepingBMP_BaseRisk,SWPlan_BaseRisk,BMPInspectionDeficiency_Rate,PEP_BaseRisk,BMP_BaseRisk,Controllable_BaseRisk,BMP_Total
0,1,150.003019,1.311222,0.963550,0.337498,1.000,1.188568,1.963550,3.152118,4.463339
1,2,150.003019,1.292442,0.982110,0.382344,1.000,0.143233,1.982110,2.125344,3.417785
2,3,150.003019,1.354261,0.394759,0.394759,0.000,0.206963,0.394759,0.601722,1.955983
3,4,150.003019,1.054281,0.359765,0.359765,0.000,0.052883,0.359765,0.412648,1.466929
4,5,150.003019,1.289362,1.091404,0.428664,0.000,0.846106,1.091404,1.937510,3.226872
5,6,150.003019,1.287252,0.351029,0.351029,0.000,3.015601,0.351029,3.366630,4.653882
6,7,150.003019,1.271192,0.452946,0.452946,0.855,0.003285,1.307946,1.311231,2.582423
7,8,150.003019,1.350908,0.339225,0.339225,0.000,0.049472,0.339225,0.388697,1.739606
8,9,150.003019,1.002867,0.323702,0.323702,0.000,1.389276,0.323702,1.712978,2.715845
9,10,150.003019,1.000678,0.351230,0.351230,0.000,0.848977,0.351230,1.200207,2.200885


# BMP FEASIBILITY EVALUATION
Talk about it...

Global variables related to existing sampling data include:  
 - 
 - 

Defined several functions that will be used by BMP Option Evaluation. These include:  
 - 
 - 

In [26]:
# %%capture cap --no-stderr
'''
#############################################################################################################
#                    EVALUATE BASE BMP FEASIBILITY at each facility  
#                Write results to the base_bmp_feasibility_test_results table.
############################################################################################################ 

'''
print('\n******Evaluating Base BMP feasibility at facilities.******')
ShowCalculations = True #flag indicating if steps should be outputted
Expr.ResetEvalErrorCount() #RESET EXPRESION EVALUATOR ERROR COUNT

#Only analyze bmps at facilities we have data for. make list of these facilities.
ls_id = [np.asscalar(id) for id in pd_exPEP_sum['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
for aFac in session.query(Facility_Chars).filter(Facility_Chars.id.in_(ls_id)):    
    if ShowCalculations: print ('\n***Evaluating base bmp feasibiilty tests for facility: ', aFac.Fac_Name), ' ***'
    myBMPs = session.query(Base_BMPs)
    for aBMP in myBMPs:
        if ShowCalculations:print ('\n######Evaluating feasibility of base_bmp: ', aBMP.bmp_name, ' ID: ', aBMP.id, '######')
        BBMP_Eval.Eval_base_bmp_feasibility_tests(aFac, aBMP, ShowCalculations)
session.commit
winsound.Beep(250,1000)
print ('*****************************************************************')
print ('* Completed evaluating Base BMP feasibility                     *')
if Expr.CountEvalErrors() >0:
    print (Expr.CountEvalErrors(), ' errors were encountered. Review output to identify location(s)')
    print ('Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:')
else:
    print ('No errors detected.')
print ('*****************************************************************')

# with open('C:\\Users\\JonHonda\\Desktop\\Crap\\output.txt', 'w') as f:
#     f.write(cap.stdout)
# f.close()


******Evaluating Base BMP feasibility at facilities.******

***Evaluating base bmp feasibiilty tests for facility:  Kalihi-Palama Bus & Paratransit Facility

######Evaluating feasibility of base_bmp:  Hydrodynamic Separation  ID:  1 ######

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='Yes'
  eval('Yes'=='Yes')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 1

  Attempting eval of feasibility_test ID:  13
proccessing expression: Feas-13=Pave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Pave_Area', ['Pave_Area', 'val', 'facility_chars', 'Pave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Pave_Area=867962.0
    attempting to retrieve value fo

  Wrote to base_bmp_feasibility_test_results as recordID: 11

  Attempting eval of feasibility_test ID:  2
proccessing expression: Feas-2=GW_Risk!='High'
    attempting to retrieve value for:  ('GW_Risk', ['GW_Risk', 'val', 'facility_chars', 'GW_Risk', 'id', 'FLOAT'])
       QUERY RESULT: GW_Risk='High'
  eval('High'!='High')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 12

  Attempting eval of feasibility_test ID:  5
proccessing expression: Feas-5=Soil_Type!='Rock'
    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type', 'val', 'facility_chars', 'Soil_Type', 'id', 'FLOAT'])
       QUERY RESULT: Soil_Type='Quarry'
  eval('Quarry'!='Rock')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 13

  Attempting eval of feasibility_test ID:  6
proccessing expression: Feas-6=Soil_Type!='Clay'
    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type

       dynamic expression: BMP_Size(base_bmps~bmp_size_expression_id~bmp_name) =  static expression: bmp_size_expr_Oil and Water Separators
       Reentering EvalExpr...
proccessing expression: bmp_size_expr_Oil and Water Separators=57.578*Indus_WQFR + 14.586
    attempting to retrieve value for:  ('Indus_WQFR', ['Indus_WQFR', 'val', 'facility_chars', 'Indus_WQFR', 'id', 'FLOAT'])
       QUERY RESULT: Indus_WQFR=12.96
  eval(57.578*12.96 + 14.586)=760.7968800000001
  eval(867962.0>760.7968800000001)=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 29

  Attempting eval of feasibility_test ID:  21
proccessing expression: Feas-21=Ex_Struct_BMP=='None'
    attempting to retrieve value for:  ('Ex_Struct_BMP', ['Ex_Struct_BMP', 'val', 'facility_chars', 'Ex_Struct_BMP', 'id', 'FLOAT'])
       QUERY RESULT: Ex_Struct_BMP='OWS'
  eval('OWS'=='None')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibi

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='No'
  eval('No'=='Yes')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 79

  Attempting eval of feasibility_test ID:  13
proccessing expression: Feas-13=Pave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Pave_Area', ['Pave_Area', 'val', 'facility_chars', 'Pave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Pave_Area=216424.0
    attempting to retrieve value for:  ('BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', ['BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', 'dxp', 'base_bmps', 'bmp_size_expression_id', 'bmp_name', 'FLOAT'])
     This is a dynamic expression. Query for static expre

######Evaluating feasibility of base_bmp:  Oil and Water Separators  ID:  11 ######

  Attempting eval of feasibility_test ID:  1
proccessing expression: Feas-1=OFFSITE_SD_Exist=='Yes'
    attempting to retrieve value for:  ('OFFSITE_SD_Exist', ['OFFSITE_SD_Exist', 'val', 'facility_chars', 'OFFSITE_SD_Exist', 'id', 'FLOAT'])
       QUERY RESULT: OFFSITE_SD_Exist='No'
  eval('No'=='Yes')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 102

  Attempting eval of feasibility_test ID:  13
proccessing expression: Feas-13=Pave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Pave_Area', ['Pave_Area', 'val', 'facility_chars', 'Pave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Pave_Area=216424.0
    attempting to retrieve value for:  ('BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', ['BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', 'dxp', 'base_bmps', 'bmp_size_expressio

...

  Wrote to base_bmp_feasibility_test_results as recordID: 1474

  Attempting eval of feasibility_test ID:  2
proccessing expression: Feas-2=GW_Risk!='High'
    attempting to retrieve value for:  ('GW_Risk', ['GW_Risk', 'val', 'facility_chars', 'GW_Risk', 'id', 'FLOAT'])
       QUERY RESULT: GW_Risk='Low'
  eval('Low'!='High')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 1475

  Attempting eval of feasibility_test ID:  5
proccessing expression: Feas-5=Soil_Type!='Rock'
    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type', 'val', 'facility_chars', 'Soil_Type', 'id', 'FLOAT'])
       QUERY RESULT: Soil_Type='Clay'
  eval('Clay'!='Rock')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 1476

  Attempting eval of feasibility_test ID:  6
proccessing expression: Feas-6=Soil_Type!='Clay'
    attempting to retrieve value for:  ('Soil_Type', ['Soil_Type',

    attempting to retrieve value for:  ('Pave_Area', ['Pave_Area', 'val', 'facility_chars', 'Pave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Pave_Area=665753.0
    attempting to retrieve value for:  ('BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', ['BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', 'dxp', 'base_bmps', 'bmp_size_expression_id', 'bmp_name', 'FLOAT'])
     This is a dynamic expression. Query for static expression using provided unique identifiers
       dynamic expression: BMP_Size(base_bmps~bmp_size_expression_id~bmp_name) =  static expression: bmp_size_expr_Oil and Water Separators
       Reentering EvalExpr...
proccessing expression: bmp_size_expr_Oil and Water Separators=57.578*Indus_WQFR + 14.586
    attempting to retrieve value for:  ('Indus_WQFR', ['Indus_WQFR', 'val', 'facility_chars', 'Indus_WQFR', 'id', 'FLOAT'])
       QUERY RESULT: Indus_WQFR=16.8468595
  eval(57.578*16.8468595 + 14.586)=984.5944762910001
  eval(665753.0>984.5944762910001)=True
  W

  Wrote to base_bmp_feasibility_test_results as recordID: 1576

  Attempting eval of feasibility_test ID:  10
proccessing expression: Feas-10=DS_SS_Exist=='Yes'
    attempting to retrieve value for:  ('DS_SS_Exist', ['DS_SS_Exist', 'val', 'facility_chars', 'DS_SS_Exist', 'id', 'FLOAT'])
       QUERY RESULT: DS_SS_Exist='Yes'
  eval('Yes'=='Yes')=True
  Writing to DB Feasibility Test Result: True(1)
  Wrote to base_bmp_feasibility_test_results as recordID: 1577

  Attempting eval of feasibility_test ID:  13
proccessing expression: Feas-13=Pave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Pave_Area', ['Pave_Area', 'val', 'facility_chars', 'Pave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Pave_Area=204046.0
    attempting to retrieve value for:  ('BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', ['BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', 'dxp', 'base_bmps', 'bmp_size_expression_id', 'bmp_name', 'FLOAT'])
     This is a

  FAULT!!!! Error occured while evaluating expression:  None>None :  '>' not supported between instances of 'NoneType' and 'NoneType'
  I will set evaluation to NoneType
  eval(None>None)=None
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 1597

######Evaluating feasibility of base_bmp:  Media Filtration (Pressure)  ID:  4 ######

  Attempting eval of feasibility_test ID:  9
proccessing expression: Feas-9=TFMR_Exist=='Yes'
    attempting to retrieve value for:  ('TFMR_Exist', ['TFMR_Exist', 'val', 'facility_chars', 'TFMR_Exist', 'id', 'FLOAT'])
       QUERY RESULT: TFMR_Exist=None
  eval(None=='Yes')=False
  Writing to DB Feasibility Test Result: False(0)
  Wrote to base_bmp_feasibility_test_results as recordID: 1598

  Attempting eval of feasibility_test ID:  10
proccessing expression: Feas-10=DS_SS_Exist=='Yes'
    attempting to retrieve value for:  ('DS_SS_Exist', ['DS_SS_Exist', 'val', 'facility_chars', 'DS_SS_Exist', 'id

  Wrote to base_bmp_feasibility_test_results as recordID: 1636

  Attempting eval of feasibility_test ID:  13
proccessing expression: Feas-13=Pave_Area>BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)
    attempting to retrieve value for:  ('Pave_Area', ['Pave_Area', 'val', 'facility_chars', 'Pave_Area', 'id', 'FLOAT'])
       QUERY RESULT: Pave_Area=None
    attempting to retrieve value for:  ('BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', ['BMP_Size(base_bmps~bmp_size_expression_id~bmp_name)', 'dxp', 'base_bmps', 'bmp_size_expression_id', 'bmp_name', 'FLOAT'])
     This is a dynamic expression. Query for static expression using provided unique identifiers
       dynamic expression: BMP_Size(base_bmps~bmp_size_expression_id~bmp_name) =  static expression: bmp_size_expr_Media Filtration (Pressure)
       Reentering EvalExpr...
proccessing expression: bmp_size_expr_Media Filtration (Pressure)=586.06*Indus_WQFR - 41.868
    attempting to retrieve value for:  ('Indus_WQFR', ['I

In [27]:
'''
#############################################################################################################
#                           Make all combinations of base bmps  
#                     Write results to the combos bmp database table
############################################################################################################ 
#MAXIMUM POLLUTANT REMOVAL RATES ARE DETERMINED BY IDENTIFYING 
#  THE BASE_BMP IN THE COMBO THAT PROVIDES THE HIGHEST REMOVAL RATE FOR A GIVEN POLLUTANT
'''
import time
print ('get a coffee...this one takes a while!')
start_time = time.time()
CBMP_Eval.Make_ALL_bmp_base_option_combos()
session.commit()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))

get a coffee...this one takes a while!
 Making BMP Combos of length: 1
 Find max pollutant removal rates for each BMP Combo of length:  1
  Made  13  combos
 Making BMP Combos of length: 2
 Find max pollutant removal rates for each BMP Combo of length:  2
  Made  78  combos
 Making BMP Combos of length: 3
 Find max pollutant removal rates for each BMP Combo of length:  3
  Made  286  combos
 Making BMP Combos of length: 4
 Find max pollutant removal rates for each BMP Combo of length:  4
  Made  715  combos
 Making BMP Combos of length: 5
 Find max pollutant removal rates for each BMP Combo of length:  5
  Made  1287  combos
 Making BMP Combos of length: 6
 Find max pollutant removal rates for each BMP Combo of length:  6
  Made  1716  combos
 Making BMP Combos of length: 7
 Find max pollutant removal rates for each BMP Combo of length:  7
  Made  1716  combos
 Making BMP Combos of length: 8
 Find max pollutant removal rates for each BMP Combo of length:  8
  Made  1287  combos
 Making

In [167]:
'''
#############################################################################################################
#                           TUNE BMP FEASIBILITY
#           
############################################################################################################ 
CODE TO TUNE BMP FEASIBILITY/.
Use base bmp feasibility results for each facility.
'''
import itertools     #https://docs.python.org/3/library/itertools.html    
import pandas as pd

from sqlalchemy import and_

def _Make_bmp_fingerprint(base_BMP_components):
    #create fingerprint of the passed list of base_bmp_ids
    #fingerprint is just a | separated list of ids of the base bmps that make up the combo bmp
    #corresponds to bmp_options table's bmp_fingerprint field
    #FORMAT: |bmp_option_base_component_id||bmp_option_base_component_id| w/ id's given in ascending order
    fingerprint = '|' + '|'.join(str(id) + '|' for id in base_BMP_components)
    return fingerprint

def Eval_FacBMPCombo(pd_basebmps, myFacility, bmpCombo):
    '''
    input:
        pdbasebmps: pandas built from a BBMP_Eval.evalFacility_BaseBMP dictionary list
                    assme that pandas is passed in w/ index is set as base_bmp_id
        myFacility: SQLA fac_chars record
        bmpCombo: list of base_bmp_ids that make up this combo
    
    #retrieve previously computed combo removal rate
    #calculate combo cip and om cost, insert/update database
    #insert/update combo data to Combo_BMP_Feasibility_Test_Results table

    '''    
    #get combo bmp pollutant removal rates into pandas 
    q = session.query(Combo_BMPs.bmp_fingerprint, Combo_BMPs.id.label('combos_bmp_id'), PRR.id.label('PRR_id'),
          PRR.r_tss, PRR.r_turbidity, PRR.r_p, PRR.r_n, PRR.r_nn, PRR.r_an,
          PRR.r_og, PRR.r_cu, PRR.r_zn, PRR.r_fe, PRR.r_phmin, PRR.r_phmax
        ).filter(Combo_BMPs.bmp_fingerprint == _Make_bmp_fingerprint(bmpCombo)).filter(
        Combo_BMPs.bmp_option_removal_rate_id == PRR.id)  
    pd_rr = pd.read_sql(q.statement,session.bind).applymap(lambda el: 0.00 if el is None else el) #el = 0. if nonetype to represent no removal rate change

    #get costs in pandas
    sumCIP = sum(pd_basebmps.loc[bmp_id,'calc_cip_cost'] for bmp_id in bmpCombo)
    sumOM = sum(pd_basebmps.loc[bmp_id,'calc_om_cost'] for bmp_id in bmpCombo)
    pd_sums = pd.DataFrame([{'calc_cip_cost':sumCIP, 'calc_om_cost': sumOM}])

    #prepare dataframe to return:
    pd_temp = pd.concat([pd_rr, pd_sums], axis = 1)#make one dataframe containing combo bmp's removal rates and costs:

#     this one's for testing only (won't work with non-base bmps):
    pd_temp.insert(0,'Facility_Name',myFacility.Fac_Name) #include facility id column
    pd_temp.insert(3,'BMP_Name', session.query(Base_BMPs.bmp_name).filter(Base_BMPs.id == q.first().combos_bmp_id).first())    
    return (pd_temp)
    

def Eval_FacBMPCombos(aFac, ShowCalculations):
    #a wrapper around Eval_FacBMPCombo
    print('\n***Evaluating feasible bmp combos for facility: ', aFac.Fac_Name, '***')
    print ('****Evaluating feasibile base bmps****')
    df = pd.DataFrame(BBMP_Eval.evalFacility_BaseBMP(aFac, False)).set_index('base_bmp_id')
    if ShowCalculations: display (df)   
    df = df.loc[df['is_feasible'] == 1]
    if ShowCalculations:
        print ('****These are the feasible base bmps. I\'ll use them to make combos:****')
        display (df)
    feas_ls = df.index#send feasible base bmp ids to list
    pd_temp = pd.DataFrame()

    for feas in feas_ls:
         pd_temp = pd.concat([pd_temp,Eval_FacBMPCombo(df,aFac, [feas])])            

    print ('      There are ', len(pd_temp), ' combinations of feasible Base BMPs.')
    return pd_temp
    
def Eval_All_FacBMPCombos():
    ShowCalculations = False
    print ('Evaluating feasibile BMP Options for each facility:')
    #Only analyze bmps at facilities we have data for. make list of these facilities.
    ls_id = [np.asscalar(id) for id in pd_exPEP_sum['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
    pd_BaseBMPCombos = pd.DataFrame() #create empty dataframe that we'll fill w/ the feasibile combos
    for aFac in session.query(Facility_Chars).filter(Facility_Chars.id.in_(ls_id)):        
#         Eval_FacBMPOptions(aFac, ShowCalculations)
        pd_BaseBMPCombos = pd.concat([pd_BaseBMPCombos, Eval_FacBMPCombos(aFac, ShowCalculations)])
    pd_BaseBMPCombos.reset_index(drop=True, inplace=True)
    return pd_BaseBMPCombos


#############################################################################################################
#                           EVALUATE FEASIBILITY OF BMP COMBOS 
#                   (DEFINE GLOBAL VARIABLE: pd_BaseBMPCombos)
############################################################################################################ 
start_time = time.time()
pd_BaseBMPCombos = Eval_All_FacBMPCombos()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))
display(pd_BaseBMPCombos)
# WRITE COMBOS RESULTS TO EXCEL FILE
xlsFile = 'C:\\Users\\JonHonda\\Desktop\\Crap\\Combos.xls'
print ('writing to excel file: ', xlsFile)
writer = pd.ExcelWriter(xlsFile)
pd_BaseBMPCombos.to_excel(writer,'Output')
writer.save()
session.commit()


Evaluating feasibile BMP Options for each facility:

***Evaluating feasible bmp combos for facility:  Kalihi-Palama Bus & Paratransit Facility ***
****Evaluating feasibile base bmps****
      There are  9  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facility:  Pearl City Bus Facility ***
****Evaluating feasibile base bmps****
      There are  9  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facility:  Kapaa Refuse Transfer Station ***
****Evaluating feasibile base bmps****
      There are  4  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facility:  Kawailoa Refuse Transfer Station ***
****Evaluating feasibile base bmps****
      There are  3  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facility:  Keehi Refuse Transfer Station ***
****Evaluating feasibile base bmps****
      There are  10  combinations of feasible Base BMPs.

***Evaluating feasible bmp combos for facil

,Facility_Name,bmp_fingerprint,combos_bmp_id,BMP_Name,PRR_id,r_tss,r_turbidity,r_p,r_n,r_nn,r_an,r_og,r_cu,r_zn,r_fe,r_phmin,r_phmax,calc_cip_cost,calc_om_cost
0,Kalihi-Palama Bus & Paratransit Facility,|1|,1,Hydrodynamic Separation,14,0.39,0.000,0.00,0.00,0.0,0.0,0.63,0.20,0.400,0.00,0.0,0.0,4.216363e+05,15552.000000
1,Kalihi-Palama Bus & Paratransit Facility,|2|,2,Enhanced Media Filtration (Replaceable Cartridge),15,0.80,0.650,0.59,0.50,0.0,0.0,0.75,0.50,0.610,0.00,0.0,0.0,2.471439e+06,18144.000000
2,Kalihi-Palama Bus & Paratransit Facility,|3|,3,Biofiltration (Vault),16,0.80,0.270,0.64,0.34,0.0,0.0,0.90,0.40,0.605,0.00,0.0,0.0,4.100963e+06,15552.000000
3,Kalihi-Palama Bus & Paratransit Facility,|4|,4,Media Filtration (Pressure),17,0.90,0.900,0.00,0.00,0.0,0.0,0.70,0.86,0.850,0.00,0.0,0.0,4.720602e+06,18144.000000
4,Kalihi-Palama Bus & Paratransit Facility,|6|,6,Inlet Insert Unit,19,0.80,0.000,0.00,0.00,0.0,0.0,0.80,0.00,0.000,0.00,0.0,0.0,7.500000e+03,8100.000000
5,Kalihi-Palama Bus & Paratransit Facility,|7|,7,Sand Filtration,20,0.70,0.780,0.33,0.21,0.0,0.0,0.00,0.00,0.450,0.45,0.0,0.0,2.729241e+06,95322.900000
6,Kalihi-Palama Bus & Paratransit Facility,|8|,8,Coagulation Enhanced Treatment,21,0.95,0.955,0.00,0.00,0.0,0.0,0.93,0.95,0.510,0.00,0.0,0.0,9.049300e+05,18144.000000
7,Kalihi-Palama Bus & Paratransit Facility,|9|,9,Roofing,22,1.00,1.000,1.00,1.00,1.0,1.0,1.00,1.00,1.000,1.00,0.0,1.0,2.640000e+04,0.000000
8,Kalihi-Palama Bus & Paratransit Facility,|13|,13,Detention,26,0.00,0.000,0.00,0.00,0.0,0.0,0.00,0.00,0.000,0.00,0.0,0.0,1.817161e+06,10591.433333
9,Pearl City Bus Facility,|1|,1,Hydrodynamic Separation,14,0.39,0.000,0.00,0.00,0.0,0.0,0.63,0.20,0.400,0.00,0.0,0.0,4.223240e+05,15579.862812


writing to excel file:  C:\Users\JonHonda\Desktop\Crap\Combos.xls


In [ ]:
'''
#############################################################################################################
#                         EVALUATE FEASIBILITY OF BMP COMBOS 
#            insert/update combo data to Combo_BMP_Feasibility_Test_Results table & pd_BaseBMPCombos 
############################################################################################################ 
CODE TO TUNE BMP FEASIBILITY/.
Use base bmp feasibility results for each facility.
'''
import itertools     #https://docs.python.org/3/library/itertools.html    
import pandas as pd

from sqlalchemy import and_

def _Make_bmp_fingerprint(base_BMP_components):
    #create fingerprint of the passed list of base_bmp_ids
    #fingerprint is just a | separated list of ids of the base bmps that make up the combo bmp
    #corresponds to bmp_options table's bmp_fingerprint field
    #FORMAT: |bmp_option_base_component_id||bmp_option_base_component_id| w/ id's given in ascending order
    fingerprint = '|' + '|'.join(str(id) + '|' for id in base_BMP_components)
    return fingerprint

def Eval_FacBMPCombo(pd_basebmps, myFacility, bmpCombo):
    '''
    input:
        pdbasebmps: pandas built from a BBMP_Eval.evalFacility_BaseBMP dictionary list
                    assme that pandas is passed in w/ index is set as base_bmp_id
        myFacility: SQLA fac_chars record
        bmpCombo: list of base_bmp_ids that make up this combo
    
    #retrieve previously computed combo removal rate
    #calculate combo cip and om cost, insert/update database
    #insert/update combo data to Combo_BMP_Feasibility_Test_Results table

    '''    
    #get combo bmp pollutant removal rates into pandas 
    q = session.query(Combo_BMPs.bmp_fingerprint, Combo_BMPs.id.label('combos_bmp_id'), PRR.id.label('PRR_id'),
          PRR.r_tss, PRR.r_turbidity, PRR.r_p, PRR.r_n, PRR.r_nn, PRR.r_an,
          PRR.r_og, PRR.r_cu, PRR.r_zn, PRR.r_fe, PRR.r_phmin, PRR.r_phmax
        ).filter(Combo_BMPs.bmp_fingerprint == _Make_bmp_fingerprint(bmpCombo)).filter(
        Combo_BMPs.bmp_option_removal_rate_id == PRR.id)  
    pd_rr = pd.read_sql(q.statement,session.bind).applymap(lambda el: 0.00 if el is None else el) #el = 0. if nonetype to represent no removal rate change

    #get costs in pandas
    sumCIP = sum(pd_basebmps.loc[bmp_id,'calc_cip_cost'] for bmp_id in bmpCombo)
    sumOM = sum(pd_basebmps.loc[bmp_id,'calc_om_cost'] for bmp_id in bmpCombo)
    pd_sums = pd.DataFrame([{'calc_cip_cost':sumCIP, 'calc_om_cost': sumOM}])

    #prepare dataframe to return:
    pd_temp = pd.concat([pd_rr, pd_sums], axis = 1)#make one dataframe containing combo bmp's removal rates and costs:
   
    return (pd_temp)
    

def Eval_FacBMPCombos(aFac, ShowCalculations):
    #a wrapper around Eval_FacBMPCombo
    print('\n***Evaluating feasible bmp combos for facility: ', aFac.Fac_Name, '***')
    print ('****Evaluating feasibile base bmps****')
    df = pd.DataFrame(BBMP_Eval.evalFacility_BaseBMP(aFac, False)).set_index('base_bmp_id')
    if ShowCalculations: display (df)   
    df = df.loc[df['is_feasible'] == 1]
    if ShowCalculations:
        print ('****These are the feasible base bmps. I\'ll use them to make combos:****')
        display (df)
    feas_ls = df.index#send feasible base bmp ids to list
    pd_temp = pd.DataFrame()

    for CBOLen in range (1, len(feas_ls)+1): #+1 so it's inclusive of last count
        for combo in  itertools.combinations(feas_ls,CBOLen):
            pd_temp = pd.concat([pd_temp,Eval_FacBMPCombo(df,aFac, list(combo))])
            if ShowCalculations:
                print ('Here is a summary of the combo: ', list(combo))
                display(Eval_FacBMPCombo(df,aFac, list(combo)))    

    print ('      There are ', len(pd_temp), ' combinations of feasible Base BMPs.')
    return pd_temp
    
def Eval_All_FacBMPCombos():
    ShowCalculations = False
    print ('Evaluating feasibile BMP Options for each facility:')
    #Only analyze bmps at facilities we have data for. make list of these facilities.
    ls_id = [np.asscalar(id) for id in pd_exPEP_sum['Facility_ID']] #id given as numpy int. cast to python int https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types
    pd_BaseBMPCombos = pd.DataFrame() #create empty dataframe that we'll fill w/ the feasibile combos
    for aFac in session.query(Facility_Chars).filter(Facility_Chars.id.in_(ls_id)):        
#         Eval_FacBMPOptions(aFac, ShowCalculations)
        pd_BaseBMPCombos = pd.concat([pd_BaseBMPCombos, Eval_FacBMPCombos(aFac, ShowCalculations)])
    pd_BaseBMPCombos.reset_index(drop=True, inplace=True)
    return pd_BaseBMPCombos


#############################################################################################################
#                           EVALUATE FEASIBILITY OF BMP COMBOS 
#                   (DEFINE GLOBAL VARIABLE: pd_BaseBMPCombos)
############################################################################################################ 
start_time = time.time()
pd_BaseBMPCombos = Eval_All_FacBMPCombos()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))
display(pd_BaseBMPCombos)
# WRITE COMBOS RESULTS TO EXCEL FILE
xlsFile = 'C:\\Users\\JonHonda\\Desktop\\Crap\\Combos.xls'
print ('writing to excel file: ', xlsFile)
writer = pd.ExcelWriter(xlsFile)
pd_BaseBMPCombos.to_excel(writer,'Output')
writer.save()
session.commit()

# BMP FEASIBILITY EVALUATION
Talk about it...

Global variables related to existing sampling data include:  
 - 
 - 

Defined several functions that will be used by BMP Option Evaluation. These include:  
 - 
 - 

In [ ]:
'''
ESTIMATE FACIITY PEP RISK REDUCTION BY USING CERTAIN BMP COMBOS
'''
import random
def CalWRS_PEP_Reduction(pd_Reduction):
    pd_return = pd_Reduction * pdWRS_PEP

FacGroup = pd_BaseBMPCombos.groupby('Facility_ID') #group combo options by facility_id
pd_BMPOpt = FacGroup.apply(lambda aFac:  aFac.iloc[random.randint(0,aFac.shape[0]-1)]) #randomly select a combo option for each facilityp


In [ ]:
# session.close()
# engine.dispose()

# This is scratch paper...

In [ ]:
# http://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
import pandas as pd #import in pandas library
print ('#get csv data and read into pandas')
df1=pd.read_csv("http://pythonhow.com/wp-content/uploads/2016/01/Income_data.csv")
print (df1)
print ('#write new dataframe w/ index set to the "State" column in the csv')
df2=df1.set_index("State").copy()
print (df2)
print ('#extract a portion of the dataframe: States = Alaska to Arkansas; and Dates 2005:2007')
print (df2.loc["Alaska":"Arkansas","2005":"2007"])

print ('Get only certain States, using a list of states:')
getStates = ['Alaska', 'Arizona']
print (df2.loc[getStates])

print ('#slice a column:')
df2.loc[: , "2005"]
print ('get a cell:')
df2.loc['Alaska','2005']
print ('#get max of 2005 data')
print (df2.loc[:,'2005'].max())
print ('take 2005 column and put into list')
LS = df2['2005'].tolist() #this is a series. we use the .tolist() to convert from series to list
print (type(LS))


In [ ]:
df = pd.DataFrame({'col1' : [1.0] * 5, 
                   'col2' : [2.0] * 5, 
                   'col3' : [3.0] * 5 }, index = range(1,6),)
display(df)
df2 = pd.DataFrame({'col1' : [10.0] * 5, 
                    'col2' : [100.0] * 5, 
                    'col3' : [1000.0] * 5 }, index = range(1,6),)
display(df2)
df.mul(df2, 0) # element by element multiplication no problems

In [ ]:
import datetime

# xmin = datetime.datetime.strptime('1/1/2018', "%m/%d/%Y").date()
# xmax = datetime.datetime.strptime('5/6/2018', "%m/%d/%Y").date()

# xmin <= datetime.date(2018,1,5) <= xmax

#     Wet Season is from: January 1 through April 30 and November 1 through December 31
#     Dry Season is from: May 1 through October 31

SampleDate = datetime.date(2018,11,1)

#Wet Season 1:
if datetime.date(SampleDate.year, 1,1) <= SampleDate <= datetime.date(SampleDate.year, 4,30):
    print ('ws 1')
elif datetime.date(SampleDate.year, 5,1) <= SampleDate <= datetime.date(SampleDate.year, 10,31):
    print ('dry')
else:
    print ('ws 2')
    
    
import numpy as np    
# np.max([float('nan'),0])
np.max([0,float('nan')])

if math.isnan(10)

In [ ]:
#import the pandas library
import pandas as pd

ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)

display (df)

display (df.groupby('Team').groups)


import numpy as np


grouped = df.groupby('Year')
print (grouped['Points'].agg(np.max))

In [ ]:
df = pd.DataFrame({'col1': [1, 2], 'col2': [0.1, 0.2]},
                      index=['a', 'b'])
for idx in df.index:
    print (idx)
    print (df.loc[idx]['col2'])

In [ ]:
import pandas as pd
import numpy as np
data = {'name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'], 
        'year': [2012, 2012, 2013, 2014, 2014], 
        'reports': [4, 24, 31, 2, 3],
        'coverage': [25, 94, 57, 62, 70]}
df = pd.DataFrame(data, index = ['Cochice', 'Pima', 'Santa Cruz', 'Maricopa', 'Yuma'])
display(df)

dfx = df.copy()

capitalizer = lambda x: x.upper()

dfx['name'] = dfx['name'].apply(capitalizer)
display(df)
display(dfx)

In [ ]:
def div0( a, b ):
    """ ignore / 0, div0( [-1, 0, 1], 0 ) -> [0, 0, 0] """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide( a, b )
        c[ ~ np.isfinite( c )] = 0  # -inf inf NaN
    return c

div0( [-1, 0, 1], 0 )


In [ ]:
afile = open("testfile.txt","a") 
# file.write("Hello World") 
# file.write("This is our new text file") 
# file.write("and this is another line.") 
# file.write("Why? Because we can.") 
print ("test", file=afile)
afile.close() 

In [ ]:
# import pandas as pd
# x = pd.DataFrame({0: [1,2,float('nan')], 1: [4,5,6], 2: [7,8,9] })
# y = pd.Series([-1, float('nan'), -1])
# print (y)
# # display(x.apply(lambda col: y*col))

# q = session.query(
#       PRR.r_tss, PRR.r_turbidity, PRR.r_p, PRR.r_n, PRR.r_nn, PRR.r_an,
#       PRR.r_og, PRR.r_cu, PRR.r_zn, PRR.r_fe, PRR.r_phmin, PRR.r_phmax
# ).limit(1)
# pd_rr = pd.read_sql(q.statement,session.bind).applymap(lambda el: 0 if el is None else el) #change nonetypes to 0 rem
# display(pd_rr)
# ls_c = ['c_' + Constituent for Constituent in pollLS] #make list of concentration headers 
# pd_c = pd_ExConcs.loc[pd_ExConcs['Facility_ID'] == 1, ls_c]
# display(pd_c)

# print (pd_rr['r_tss'][0])

# # pd_ConcsRed = pd_ExConcs.loc[pd_ExConcs['Facility_ID'] ==155, ['Facility_ID', 'sample_date']]
# # for Constituent in pollLS:
# #     pd_ConcsRed['c_' + Constituent] = pd_c['c_' + Constituent] * pd_rr['r_' + Constituent][0]



# display(pd_ConcsRed)

In [134]:
#WHICH METHOD IS FASTER? APPLY OR LIST COMPREHENSION?
num = 10000
start_time = time.time()
def fun1a(x):
    return pd.DataFrame([{'fun1':x.loc['id']}])
pd_maina = pd.DataFrame([{'id': i} for i in range(0,num)])
pd_maina['fun1'] = pd_maina.apply(lambda x: fun1a(x), axis=1)
print ('--- %s execution time in seconds ---' % (time.time() - start_time))
# display(pd_maina)

start_time = time.time()
def fun1b(x):
    return pd.DataFrame([{'fun1':x}])
pd_mainb = pd.DataFrame([{'id': i} for i in range(0,num)])
ls = [fun1b(x) for x in range(0,num)]
pd_tmp = pd.concat(ls)
pd_tmp.reset_index(drop=True, inplace=True)
# display(pd_mainb)
# display(pd_tmp)
g = pd.concat([pd_mainb,pd_tmp], axis=1)
print ('--- %s execution time in seconds ---' % (time.time() - start_time))
# display(g)

# display(pd_main)

--- 5.5959999561309814 execution time in seconds ---
--- 5.3989996910095215 execution time in seconds ---


ValueError: Shape of passed values is (3, 103), indices imply (3, 101)